# Evaluation

## Dataset

In [0]:
%run "./0. Init"

In [0]:
dbutils.library.restartPython()

In [0]:
import mlflow
import pandas as pd
from chain import chain, vector_search_as_retriever
from mlflow.metrics.genai import relevance, faithfulness, EvaluationExample

In [0]:
questions = [
    "How can I install an ecometer?",
    "How to replace brake pads?",
    "Is there a way to determine which axle I have to install the axle beam on?",
    "What are the steps needed to install a new brake caliper?",
]
sources = [
    'dbfs:/Volumes/test-catalog/bronze/volume_hackathon/pdfs/Brake_Drum-HKN-Workshop_manual-BPW-2024-_35192401e.pdf',
    'dbfs:/Volumes/test-catalog/bronze/volume_hackathon/pdfs/Trailer_Disc_brake_TSB-ECODisc-_Workshop_manual_2023_en-BPW-35292301e.pdf',
    'dbfs:/Volumes/test-catalog/bronze/volume_hackathon/pdfs/HKN_trailer_axles_Spare_parts_list_BPW_31022101e_2021.pdf',
    'dbfs:/Volumes/test-catalog/bronze/volume_hackathon/pdfs/Trailer_Disc_brake_TSB-ECODisc-_Workshop_manual_2023_en-BPW-35292301e.pdf'
]

eval_df = pd.DataFrame({"question": questions, "source": sources})

In [0]:
def model(input_df):
    import json

    answer = []
    for index, row in input_df.iterrows():
        outputs = chain.invoke({"messages": [{"role": "user", "content": row["question"]}]})
        out_dict = json.loads(outputs) if type(outputs) == str else outputs

        # Rename output keys for Mosaic AI evaluation
        del out_dict['question']
        out_dict['retrieved_context'] = out_dict.pop('references')
        out_dict['response'] = out_dict.pop('answer')
        
        answer.append(out_dict)

    return answer

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lakehouse 0.20.0 requires packaging==24.1, but you have packaging 23.2 which is incompatible.
lakehouse 0.20.0 requires requests==2.32.3, but you have requests 2.31.0 which is incompatible.
lakehouse 0.20.0 requires urllib3==2.2.2, but you have urllib3 2.0.7 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.2.5 requires packaging<24.0,>=23.2, but you have packaging 24.1 which is incompatible.
mlflow 2.16.2 requires mlflow-skinny==2.16.2, but you have mlflow-skinny 2.17.0 which is incompatible.
pdfservices-sdk 4.0.0 requires request

In [0]:
def generate_eval_set_from_answer(questions, answers, to_pandas: bool = False):
    examples =  {
        "request": questions,
        # "expected_response": [ # Optional, needed for judging correctness.
        #     "Spark is a data analytics framework.",
        #     "To convert a Spark DataFrame to Pandas, you can use the toPandas() method.",
        # ]
    }
    responses = []
    retrieved_contexts = []
    for answer in answers:
        contents = []
        doc_uris = []
        for context in answer['retrieved_context']:
            contents.append(context['content'])
            doc_uris.append(context['doc_uri'])
        retrieved_contexts.append([dict(zip(['doc_uri', 'content'], [uri, content])) for uri, content in zip(doc_uris, contents)])
        responses.append(answer['response'])

    examples['retrieved_context'] = retrieved_contexts
    examples['response'] = responses

    return examples if not to_pandas else pd.DataFrame(examples)

## Evaluation of Retrieval system with MLflow

In [0]:
from typing import List

def evaluate_retriever():

    def retrieve_doc_urls(question: str) -> List[str]:
        docs = vector_search_as_retriever.get_relevant_documents(question)
        doc_urls = [doc.metadata["url"] for doc in docs]
        return doc_urls

    def retriever_model_function(question_df: pd.DataFrame) -> pd.Series:
        return question_df["question"].apply(retrieve_doc_urls)
    
    mlflow.autolog()
    with mlflow.start_run() as run:
        evaluate_results = mlflow.evaluate(
            model=retriever_model_function,
            data=eval_df,
            model_type="retriever",
            targets="source",
            evaluators="default",
        )
    return evaluate_results

embedding_results = evaluate_retriever()

2024/10/23 10:24:22 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/10/23 10:24:24 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/10/23 10:24:37 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/10/23 10:24:37 WARNING mlflow.metrics.metric_definitions: Cannot calculate metric 'precision_at_k' for non-arraylike of string or int inputs. Non-arraylike of strings/ints found for the column specified by the `targets` parameter on row 0, value dbfs:/Volumes/test-catalog/bronze/volume_hackathon/pdfs/Brake_Drum-HKN-Workshop_manual-BPW-2024-_35192401e.pdf. Skipping metric logging.
2024/10/23 10:24:37 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'precision_at_3' because it returned None.
2024/10/23 10:24:37 WARNING mlflow.metrics.metric_definitions: Cannot calculate metric 'recall_at_k' for non-arraylike of string or int inputs. Non-arraylike of strings/ints fo

[Trace(request_id=tr-55446a0596b84422ab263e31dd1a14ba), Trace(request_id=tr-d518e713457d432cbc617a93c08ef442), Trace(request_id=tr-a4412b523e0d4ed5bf81acdf3a1b494a), Trace(request_id=tr-e96c2b5e705e4ba7a10507af64fc742e)]

In [0]:
eval_results_of_retriever = embedding_results.tables["eval_results_table"]
display(eval_results_of_retriever)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File <command-3521381254536335>, line 1
----> 1 eval_results_of_retriever = embedding_results.tables["eval_results_table"]
      2 display(eval_results_of_retriever)

KeyError: 'eval_results_table'

## Evaluation of RAG app with MosaicAI using LLM-as-a-judge

In [0]:
mlflow.langchain.autolog()
with mlflow.start_run(run_name='hackathon_rag_evalutation'):
    answers = model(eval_df)
    result = mlflow.evaluate(
        data=generate_eval_set_from_answer(questions, answers, to_pandas=True),    # Your evaluation set
        # model=model, # If you have an MLFlow model. `retrieved_context` and `response` will be obtained from calling the model.
        model_type="databricks-agent",  # Enable Mosaic AI Agent Evaluation
    )

Evaluating:   0%|          | 0/4 [Elapsed: 00:00, Remaining: ?]

<!DOCTYPE html>
 
 
 Evaluation output 
 
 
 
 
 
 
 
 <!-- AI icon -->
 
 
 <path fill="currentColor" fill-rule="evenodd"
 d="M4 9a.75.75 0 0 1 .739.621l.152.877a.75.75 0 0 0 .61.61l.878.153a.75.75 0 0 1 0 1.478l-.877.152a.75.75 0 0 0-.61.61l-.153.878a.75.75 0 0 1-1.478 0l-.152-.877a.75.75 0 0 0-.61-.61l-.878-.153a.75.75 0 0 1 0-1.478l.877-.152a.75.75 0 0 0 .61-.61l.153-.878A.75.75 0 0 1 4 9Zm0 2.92a2.289 2.289 0 0 1-.08.08c.028.026.054.052.08.08.026-.028.052-.054.08-.08a2.289 2.289 0 0 1-.08-.08ZM10 0c.36 0 .67.257.737.611l.264 1.398A3.75 3.75 0 0 0 13.99 5l1.398.264a.75.75 0 0 1 0 1.474l-1.398.264A3.75 3.75 0 0 0 11 9.99l-.264 1.398a.75.75 0 0 1-1.474 0l-.264-1.398A3.75 3.75 0 0 0 6.01 7l-1.398-.264a.75.75 0 0 1 0-1.474l1.398-.264A3.75 3.75 0 0 0 9 2.01L9.263.611A.75.75 0 0 1 10 0Zm0 3.682A5.256 5.256 0 0 1 7.682 6 5.256 5.256 0 0 1 10 8.318 5.256 5.256 0 0 1 12.318 6 5.256 5.256 0 0 1 10 3.682Z"
 clip-rule="evenodd"> 
 
 Evaluation completed: 2/4 pass all assessments 
 
 
 
 
 
 
 
 
 
 <!-- "Open in a new tab" icon -->
 
 
 
 
 View evaluation results
 
 
 
 <!-- "Open in a new tab" icon -->
 
 
 
 
 Compare evaluations across runs
 
 
 
 
 Python APIs: 
 
 mlflow.evaluate() returns a result object that is an instance of EvaluationResult . 
 
 
 result.metrics : A dictionary of aggregated evaluation results.
 
 
 result.tables['eval_results'] : A DataFrame of per-request evaluation results.
 
 

 
 For more details, see the mlflow.evaluate 
 API reference or the MLflow
 LLM Evaluate Quickstart .

2024/10/23 08:50:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run hackathon_rag_evalutation at: https://adb-4176489020682060.0.azuredatabricks.net/ml/experiments/64bb7be8d7ac4397b53ab2c7b339465c/runs/4b71b2b9f828443c94ceb1387d131bf9.
2024/10/23 08:50:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://adb-4176489020682060.0.azuredatabricks.net/ml/experiments/64bb7be8d7ac4397b53ab2c7b339465c.


[Trace(request_id=tr-e0fe15eea8954ce18d46d4ccd7850cb4), Trace(request_id=tr-24eaa5b658e0452cb17f225b8c3d4f67), Trace(request_id=tr-93811cd69be44c0fabd46dfffe9c5595), Trace(request_id=tr-da05e05b18cd446c8dbb6acea24c59c3)]

In [0]:
eval_results = result.tables["eval_results"]
display(eval_results)

request_id request response expected_retrieved_context expected_response retrieved_context grading_notes trace response/overall_assessment/rating response/llm_judged/safety/rating response/llm_judged/safety/rationale response/llm_judged/relevance_to_query/rating response/llm_judged/relevance_to_query/rationale response/llm_judged/groundedness/rating response/llm_judged/groundedness/rationale retrieval/llm_judged/chunk_relevance/ratings retrieval/llm_judged/chunk_relevance/rationales retrieval/llm_judged/chunk_relevance/precision 8e7ba8435088691f60e75348af88fa66e560acfc7ce4b2bda804f4707356c90b How can I install an ecometer? Here is the restructured answer:

**Tools required:**

* Torque wrench (to tighten the ECOMETER to the specified torque of 800 Nm)
* Torque controlled (not impact!) airguns (as an alternative to a torque wrench)

**Parts required:**

* New O-ring for the wheel hub
* New retaining key with integrated magnet
* New clasp locking ring with integrated magnet
* BPW special longlife grease ECO-Li Plus

**Steps required:**

1. Remove the hub cap and hooked spring ring.
2. Insert a new retaining key with integrated magnet into the recess in the axle bolt and the gearing of the toothed lock washer.
3. Insert the retaining key of the new clasp locking ring with integrated magnet into the groove between the axle stub and the nut.
4. Hook the clasp locking ring behind the edge of the axle nut, making sure to hook in the safety lock on the clasp locking ring.
5. Apply a thin coat of BPW special longlife grease ECO-Li Plus to the ECOMETER in the area of the connection thread.
6. Screw the ECOMETER onto the wheel hub and tighten to the tightening torque of 800 Nm.

**Safety advices:**

* Ensure the vehicle is securely parked: Park the vehicle on a flat, stable surface and engage the parking brake to prevent movement.
* Wear protective gear: Use gloves, safety glasses, and sturdy work boots to protect your hands, eyes, and feet.
* Use proper lifting equipment: Use a jack rated for the weight of the vehicle and jack stands to support it safely before removing the wheel.
* Avoid contact with hot components: Let the brakes cool down completely before starting the work to avoid burns from hot parts.
* Work in a well-ventilated area: If applying grease, work in a well-ventilated area to avoid inhaling fumes.
* Handle grease carefully: Grease is slippery; avoid skin contact and clean spills immediately.
* Check for correct tool usage: Use the correct torque wrench to avoid stripping the bolts.
* Do not over-tighten the ECOMETER: Ensure the tightening torque of 800 Nm is not exceeded to avoid damage to the ECOMETER or wheel hub.
* Check ECOMETER function before use: After reassembly, check the ECOMETER function to ensure proper installation and check for any unusual noises or vibrations before moving the vehicle. null null List(List(BPW-WH-HKN 35192401e Page 133
 Repair guide!
The ECOMETER is not allowed to be installed, removed or set in a poten-
tially explosive atmosphere.
Renew the unit in case of damage to 
the housing, the battery compartment or battery cables, the casting com-
pound or other damage to the unit.
Protect the reed contact in the cap 
against damage. Do not stack the caps one inside the other, for example. 
The ECOMETER for ECO Plus 2 axles 
with bayonet lock must be mounted with a new O-ring.
The ECOMETER for ECO
Plus axles must 
be ﬁ tted and dismantled using only 
torque controlled (not impact!) airguns or manually with a torque wrench.Do not bend the magnet with the spring when installing and removing.Picture 2
Picture 316.1 Function
 A minicomputer that is protected from water and 
dirt counts the wheel revolutions using a magnet and a reed contact. 
 The tyre rolling circumference is set initially and 
cannot be changed subsequently.
 There is a sensed area behind the display disc 
which is indicated by the two protruding BPW logos. The display is activated by bringing a metallic object (ferromagnetic) clo